# Coffee Machine Power Roll Up

Link to the example in the Cookbook:\
[https://mms.openmbee.org/alfresco/mmsapp/mms.html#/projects/PROJECT-ID_2_[...]5/views/_19_0_2_f46036d_1582648674041_192673_2975](https://mms.openmbee.org/alfresco/mmsapp/mms.html#/projects/PROJECT-ID_2_2_17_7_33_25_AM_3ccfaf88_159fe0d7ba9__7d4e_cae_tw_jpl_nasa_gov_127_0_0_1/master/documents/_18_5_2_e64033a_1513278979739_556269_28755/views/_19_0_2_f46036d_1582648674041_192673_2975)

## Structure & Behavior
![img/coffee_structure.svg](img/coffee_structure.svg)
![img/ed247dde-6f79-46f4-9b1d-1e0fae300c9b1584727457366..svg](img/ed247dde-6f79-46f4-9b1d-1e0fae300c9b1584727457366..svg)
![img/5d5a481a-8f40-461e-b9fd-9ea6cc8b3db81584727470124..svg](img/5d5a481a-8f40-461e-b9fd-9ea6cc8b3db81584727470124..svg)
![img/bb4c84ec-fe4d-405f-8621-4e428270d2de1583425517408..svg](img/bb4c84ec-fe4d-405f-8621-4e428270d2de1583425517408..svg)
## Power Analysis
![img/coffee_analysis.svg](img/coffee_analysis.svg)
## Power Analysis Context
![img/coffee_constraint.svg](img/coffee_constraint.svg)

In [8]:
package 'CoffeeMaker Dynamic Power Roll-Up'{
    import ScalarValues::Real;
    import ScalarFunctions::sum;
    
    import 'Power Roll-Up Pattern'::*;
    import Behavior::*;
    import Structure::*;
    
    value type Power specializes Real;
    
    package 'Power Roll-Up Pattern' {
        
        block 'PowerRollUpPattern'{
            part subPower : 'PowerRollUpPattern'[*];
            
            value 'totalPower' : Power = sum(subPower::totalPower) + drawPower::power_draw;
            
            perform action drawPower : DrawPower;
        }

        activity DrawPower{
            value power_draw : Power;
        }

    }
    
    package Behavior {
        activity 'Make Coffee'(){
            first start;
            then action grindBeans : GrindBeans;
            then action heatWater : HeatWater;
            then action brewCoffee : BrewCoffee;
            then action keepWarm : KeepWarm;
            then done;
        }

        activity GrindBeans specializes DrawPower {
            value redefines power_draw = 5;
        }
        activity HeatWater specializes DrawPower {
            value redefines power_draw = 10;
        }
        activity BrewCoffee specializes DrawPower {
            value redefines power_draw = 2;
            
            first start;
            then action filterCoffee : FilterCoffee;
            then done;
        }
        activity FilterCoffee specializes DrawPower {
            value redefines power_draw = 1;
        }
        activity KeepWarm specializes DrawPower {
            value redefines power_draw = 1;
        }
    }
    
    package Structure {
        block 'Coffee Machine' specializes PowerRollUpPattern{
            
            perform action 'make coffee' : 'Make Coffee' redefines drawPower;
            
            part 'coffee Grinder' : 'Coffee Grinder' subsets subPower {
                perform 'make coffee'::grindBeans redefines drawPower;
            }
            part 'coffee Pot' : 'Coffee Pot' subsets subPower {
                perform 'make coffee'::heatWater redefines drawPower;
            }
            part brewer : 'Brewer' subsets subPower {
                perform 'make coffee'::brewCoffee redefines brewCoffee;
            }
            part 'water Heater' : 'Water Heater' subsets subPower {
                perform 'make coffee'::keepWarm redefines drawPower;
            }            
        }
        
        block 'Coffee Grinder'  specializes 'Power Roll-Up Pattern'::'PowerRollUpPattern';
        
        block 'Coffee Pot' specializes 'Power Roll-Up Pattern'::'PowerRollUpPattern';
        
        block 'Brewer' specializes 'Power Roll-Up Pattern'::'PowerRollUpPattern'{
            perform action brewCoffee : BrewCoffee redefines drawPower;
            
            part filter : 'Filter' subsets subPower {
                perform brewCoffee::filterCoffee redefines drawPower;
            }
        }
        
        block 'Water Heater' specializes 'Power Roll-Up Pattern'::'PowerRollUpPattern';
        
        block 'Filter' specializes 'Power Roll-Up Pattern'::'PowerRollUpPattern';
    }
    
    package 'Power Analysis' {     
        block 'Power Analysis Context'{
            value SIMULATION_TIME : Real;
            
            value totalPower : Power = 'coffee Machine'::totalPower;
            value power_per_time : Power = 'coffee Machine'::totalPower/SIMULATION_TIME;
            
            part 'coffee Machine' : 'Coffee Machine';           
        }
    }
}

Package CoffeeMaker Dynamic Power Roll-Up (a204c106-7889-4d94-b0e9-c6ac7cdf37ee)
